In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [11]:
iris=load_iris()
X=iris.data[:,2:]#taking only petal lenght and width
y=iris.target
X

array([[ 1.4,  0.2],
       [ 1.4,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.7,  0.4],
       [ 1.4,  0.3],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.5,  0.1],
       [ 1.5,  0.2],
       [ 1.6,  0.2],
       [ 1.4,  0.1],
       [ 1.1,  0.1],
       [ 1.2,  0.2],
       [ 1.5,  0.4],
       [ 1.3,  0.4],
       [ 1.4,  0.3],
       [ 1.7,  0.3],
       [ 1.5,  0.3],
       [ 1.7,  0.2],
       [ 1.5,  0.4],
       [ 1. ,  0.2],
       [ 1.7,  0.5],
       [ 1.9,  0.2],
       [ 1.6,  0.2],
       [ 1.6,  0.4],
       [ 1.5,  0.2],
       [ 1.4,  0.2],
       [ 1.6,  0.2],
       [ 1.6,  0.2],
       [ 1.5,  0.4],
       [ 1.5,  0.1],
       [ 1.4,  0.2],
       [ 1.5,  0.1],
       [ 1.2,  0.2],
       [ 1.3,  0.2],
       [ 1.5,  0.1],
       [ 1.3,  0.2],
       [ 1.5,  0.2],
       [ 1.3,  0.3],
       [ 1.3,  0.3],
       [ 1.3,  0.2],
       [ 1.6,  0.6],
       [ 1.9,  0.4],
       [ 1.4,  0.3],
       [ 1.6,  0.2],
       [ 1.4,

In [15]:
tree_clf=DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X,y)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [34]:
from sklearn.tree import export_graphviz
export_graphviz(tree_clf,out_file='iris_tree.dot',feature_names=iris.feature_names[2:],class_names=iris.target_names,
               rounded=True,filled=True)

NameError: name 'tree_clf' is not defined

In [9]:
import numpy as np
import pandas as pd

In [10]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# for i in range(1,1000):
# random.sample(range(1,100,1))
# X_train=X_train[random.sample(range(1,101),100)]
X_train.shape

(8000, 2)

In [12]:
from sklearn.model_selection import ShuffleSplit
rs=ShuffleSplit(n_splits=1000,test_size=len(X_train)-100,random_state=42)
mini_sets=[]
for i_train,i_test in rs.split(X_train):
    train=X_train[i_train]
    test=y_train[i_train]
    mini_sets.append((train,test))

In [13]:
from sklearn.tree import DecisionTreeClassifier
clf_tree=DecisionTreeClassifier(criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=17, min_impurity_split=0.0,
            min_samples_leaf=1, min_samples_split=2,
            presort=False, random_state=42,
            splitter='best')


In [14]:
from sklearn.metrics import accuracy_score
from sklearn.base import clone
forest=[clone(clf_tree) for _ in range(1000)]
accuracy_scores=[]
for tree,(x_tr,y_tr) in zip(forest,mini_sets):
    tree.fit(x_tr,y_tr)
    y_pred=tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test,y_pred))
np.mean(accuracy_scores)    

0.80544949999999993

In [15]:
Y_pred = np.empty([1000, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [16]:
from scipy.stats import mode
freq_class,class_count=mode(Y_pred,axis=0)

In [17]:
accuracy_score(y_test,freq_class.reshape([-1]))

0.872

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [19]:
log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()
voting_clf=VotingClassifier(estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],voting='hard')
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomF...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         n_jobs=1, voting='hard', weights=None)

In [20]:
for clf in [log_clf,rnd_clf,svm_clf,voting_clf]:
    clf.fit(X_train,y_train)
    print('{} - {}'.format(clf.__class__.__name__,accuracy_score(y_test,clf.predict(X_test))))
    
    

LogisticRegression - 0.8415
RandomForestClassifier - 0.849
SVC - 0.8705
VotingClassifier - 0.8645


In [21]:
log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC(probability=True)
voting_clf=VotingClassifier(estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],voting='soft')
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomF...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))],
         n_jobs=1, voting='soft', weights=None)

In [31]:
for clf in [log_clf,rnd_clf,svm_clf,voting_clf]:
    clf.fit(X_train,y_train)
    print('{} - {}'.format(clf.__class__.__name__,accuracy_score(y_test,np.argmax(clf.predict_proba(X_test),axis=1))))

LogisticRegression - 0.8415
RandomForestClassifier - 0.8495
SVC - 0.871
VotingClassifier - 0.8665


In [29]:
np.argmax(log_clf.predict_proba(X_test),axis=1)

array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

In [32]:
from sklearn.ensemble import BaggingClassifier
BaggingClassifier()

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=False, max_features=1.0, max_samples=1.0,
         n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

In [7]:
from sklearn.datasets import load_boston
X,y=load_boston(return_X_y=True)

In [23]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X,y)
gbreg=GradientBoostingRegressor(n_estimators=120)
gbreg.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=120,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [25]:
#staged_predict() is used for early stopping or find out at which state the model starts overfitting

from sklearn.metrics import mean_squared_error
errors=[mean_squared_error(y_val,y_pred) for y_pred in gbreg.staged_predict(X_val)]
print(errors)
np.argmin(errors)

[0.22384222071887061, 0.20215317097925539, 0.18464950932858054, 0.17050969524115459, 0.15870506266267148, 0.1489805706306899, 0.14058296253474759, 0.13394207298308788, 0.1282221179750854, 0.12378302225350132, 0.11999087139400676, 0.11683348789393072, 0.11411410920683354, 0.11190744122342121, 0.1099486797626025, 0.10839100776577157, 0.1071386785718882, 0.10597517058885879, 0.10498517142398218, 0.10401915539867919, 0.10321312271281204, 0.10257505565679348, 0.10195782625303913, 0.10159785328948388, 0.10109311274319401, 0.10069238173686226, 0.10022079057651342, 0.099929196452362867, 0.09961600870162321, 0.099461081531353551, 0.099217082151944511, 0.099031857985395808, 0.098805596142825788, 0.098698407174102026, 0.098558329908074976, 0.098371689794388567, 0.098126769432374819, 0.098066096358820354, 0.098001466494707729, 0.097934913156905889, 0.097837234867547307, 0.097776088388594209, 0.097714593303637687, 0.097637384697386045, 0.097596815000698278, 0.097588378256495953, 0.09758061127363777

72

In [30]:
# earlystopping without fully iterating through
mod_gb=GradientBoostingRegressor(max_depth=3,warm_start=True)
min_score=float('inf')
print(min_score)
for i in range(1,120):
    mod_gb.n_estimators=i
    mod_gb.fit(X_train,y_train)
    pred_val=mod_gb.predict(X_val)
    val_score=mean_squared_error(y_val,pred_val)
    if val_score<min_score:
        min_score=val_score
        going_up=0
    else:
        going_up+=1
    if going_up==5:
        break
print('Optimum number of estimators are {}'.format(i))
    

inf
Optimum number of estimators are 65
